In [ ]:
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from zumi.zumi import Zumi
import cv2
import IPython.display
import PIL.Image
import time
import math
import numpy as np
from scipy.stats import mode

screen = Screen()
zumi = Zumi()

In [ ]:
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from zumi.zumi import Zumi
import cv2
import IPython.display
import PIL.Image
import time
import math
import numpy as np
from scipy.stats import mode

screen = Screen()
zumi = Zumi()

#width = 1280
#height = 960

#width = 640
#height = 480

#width = 320
#height = 240

width = 160 
height = 120

#width = 80
#height = 60

camera = Camera(width,height)
camera.start_camera()

black_white_threshold = 100
minLineLength = 20
maxLineGap = 15

#given a list of lines
def find_avg_line_angle(lines,image):
    angle_thresh = 30
    avg_angle = None
    if lines is not None:
        lst = []
        cnt_x_points = []
        cnt_y_points = []
        for i in range(len(lines)):
            for x1,y1,x2,y2 in lines[i-1]: 
                radians = math.atan2((x2-x1),(y2-y1))
                degrees = math.degrees(radians)
                
                #filter the angles between 70 and 110 degrees
                if degrees > 90-angle_thresh and degrees < 90+angle_thresh: 
                    lst.append(degrees)
                    cv2.line(image,(x1,y1),(x2,y2),(255,255,255),1)
                    center_x_point = (x2-x1)/2
                    center_y_point = (y2-y1)/2
                    cnt_x_points.append(center_x_point)
                    cnt_y_points.append(center_y_point)                   
        if(len(lst)):
            avg_angle = sum(lst) / len(lst)    
        return avg_angle


try: 
    while True:

        image = camera.capture() # Take a picture 
        #focus only on the bottom half and extra of the image
        #image = image[int(height/1.5):int(height/1.5)+height, 0:width]

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert it to gray
        #gray = cv2.bilateralFilter(gray,9,75,75)
        #ret, black_white = cv2.threshold(gray,black_white_threshold,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
        black_white = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV,7,4)
        
        edges = cv2.Canny(gray, 100, 200)
        edges = cv2.dilate(edges, None)
        lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
        #make a black blank small image so you can draw white lines
        ret, blank_image = cv2.threshold(black_white,255,255,cv2.THRESH_BINARY)
        
        avg_angle = find_avg_line_angle(lines,blank_image)
            
        #print(mode(lst)[0])
        cv2.putText(blank_image, "ang:" + format(avg_angle), (int(width/2), height-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255), 1, cv2.LINE_AA)  

        #edges = cv2.dilate(edges, None)
        #find_contours(black_white)
        #IPython.display.display(PIL.Image.fromarray(gray))
        IPython.display.display(PIL.Image.fromarray(black_white))
        IPython.display.display(PIL.Image.fromarray(edges))
        IPython.display.display(PIL.Image.fromarray(blank_image))
        IPython.display.clear_output(wait=True)
        
finally:
    camera.close()
    zumi.stop()